# Titanic Genetic Model

In [1]:
import pandas as pd
import tpot

In [2]:
df = pd.read_csv("data/train.csv").set_index("PassengerId")

In [3]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Transforming dataset

### Sex column string to value

In [4]:
df["Sex"] = df["Sex"].astype('category').cat.codes

## Selecting columns for model

In [5]:
feature_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'] #'SibSp', 'Parch', 'Fare'
label = 'Survived'

## Splitting dataframe into train and test

In [6]:
df.shape

(891, 11)

In [7]:
X_train = df[:750][feature_columns]
y_train = df[:750][label]

In [8]:
X_test = df[750:][feature_columns]
y_test = df[750:][label]

## TPOT

In [9]:
gen_models = tpot.TPOTClassifier(generations=5, population_size=200, verbosity=2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
gen_models.fit(X_train, y_train)

Imputing missing values in feature set


Generation 1 - Current best internal CV score: 0.8227430552468998


Generation 2 - Current best internal CV score: 0.8254186704594279


Generation 3 - Current best internal CV score: 0.8254186704594279


Generation 4 - Current best internal CV score: 0.8280139858067767


Generation 5 - Current best internal CV score: 0.8280500170377942



Best pipeline: GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=DEFAULT, GradientBoostingClassifier__max_depth=4, GradientBoostingClassifier__max_features=0.4, GradientBoostingClassifier__min_samples_leaf=20, GradientBoostingClassifier__min_samples_split=18, GradientBoostingClassifier__n_estimators=DEFAULT, GradientBoostingClassifier__subsample=0.85)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        generations=5, max_eval_time_mins=5, max_time_mins=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=200,
        population_size=200, random_state=None, scoring=None,
        subsample=1.0, verbosity=2, warm_start=False)

In [11]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(X_test)

In [12]:
print(gen_models.score(imp.transform(X_test), y_test))

0.86524822695


In [13]:
gen_models.export('tpot_titanic_pipeline.py')